In [49]:
import numpy  as np
from pprint import pprint
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def d3_d2(d3):
    d2 = []
    for i in range(len(d3)):
        for j in range(len(d3[i])):
                d2.append(d3[i][j])
    return d2

#### Load `Iris dataset`

In [50]:
tmp = datasets.load_iris()
data_iris = tmp.data
target = tmp.target
data_train, data_test, target_train, target_test = train_test_split(data_iris, target, test_size=0.2, random_state=75)

def sanitize_data(raw,labels): #2d to 3d
    indexlabel = np.unique(labels)
    
    fixed_data = [[] for _ in range(len(indexlabel))]
    fixed_label = [[] for _ in range(len(indexlabel))]
    for i in range(len(labels)):
        fixed_data[labels[i]].append(raw[i])
        fixed_label[labels[i]].append(labels[i])
    return fixed_data, fixed_label

data_train,target_train = sanitize_data(data_train,target_train) #3d
data_test,target_test = sanitize_data(data_test, target_test)    #3d

In [51]:
class LDA:
    def calc_means(data,debug = False):
        ar = (np.array(data))
        means = []
        means_feature_wise = []
        for i in ar:
            i = np.array(i)
            rows,columns = i.shape
            means.append([np.array(np.mean(i, axis=0)) for _ in range(rows)])
            means_feature_wise.append(np.array(np.mean(i, axis=0)))
        if(debug == True):
             print("calc_means\n")
             print("array data\n",ar)
             print("rows and col",rows,columns)
             print("Means: ",means)
             print(means_feature_wise)
        return means, means_feature_wise
    def calc_Sjs(data,means,debug = False):
         features_number = (len(data[0][0]))
         Sw = [[0 for _ in range(features_number)] for _ in range(features_number)]
         count = 0
         for i in data:
            number_of_data = len(i)
            i = np.array(i).T
            mi = np.array(means[count]).T
            # rows,columns = i.shape
          
            single = (np.dot(i-mi,(i-mi).T))
            ith_sj = single/(number_of_data-1)
            Sw += (ith_sj)
            count+=1
         if(debug == True):
            print("calc_Sj\n")
            print(number_of_data-1)
            print("array data\n",data)
            print("array means\n",means)
            print("list of Sw\n",Sw)
         return Sw
    def calc_Swinverse(Sw,debug = False):
        Swinverse = np.linalg.inv(Sw)
        if(debug == True):
            print("calc_Swinverse\n")
            print("Sw\n",Sw)
            print("Swinverse\n",Swinverse)
        return Swinverse
    def calc_Sb(data,Sw,means_feature_wise,debug = False):
        n,m = Sw.shape
        # features = len(data[0][0])
        Sb = np.zeros((n,m))
        # data = np.array(data) classes*datapoint*features
        freq = [len(data[i]) for i in range(len(data))]
        total_freq = sum(freq)
        mean = 0#[0 for i in range(len(freq))] # == [10.66666667 10.5       ]

        for i in range(len(freq)): mean += (freq[i]*means_feature_wise[i])/(total_freq)
        for i in range(len(means_feature_wise)):
            mj = np.matrix(means_feature_wise[i] - mean)
            Sb += (np.multiply(freq[i],(np.dot(mj.T,mj))))
        if(debug):
            print("calc_Sb\n")
            print("data\n",data)
            print("Sw\n",Sw)
            print("Sb\n",Sb)
        return Sb
    def A(Swi,Sb, debug =False):
        if(debug) : pprint(np.dot(Swi,Sb))
        return np.dot(Swi,Sb)
    def eigen_vector(A,reduced_features : int , debug =False):
        eigenvalues,eigenvectors = np.linalg.eig(A)
        
        #sorting according to eigenvalue
        idx = eigenvalues.argsort()[::-1]
        sortedeigenvalues = eigenvalues[idx]
        sortedeigenvectors = [[] for _ in range(len(idx))]
        for i in range(0, len(idx)): sortedeigenvectors[i] = eigenvectors[idx[i]]

        if(debug == True):
            print("\n-> eigen_vector")
            print(" eigenvalue\n",eigenvalues)
            print(" eigenvector\n",eigenvectors)
            
            print("sorted eigenvalue\n",sortedeigenvalues)
            print("sorted eigenvector\n",sortedeigenvectors)
        
        return np.array(sortedeigenvectors[0:reduced_features]), np.array(sortedeigenvalues[0:reduced_features])
    def feature_reduction(data,vector,debug =False):
        reduced = []
        for i in data:
            u = []
            for j in i:
                j = np.dot(vector,np.array(j))
                u.append(j)
            reduced.append(u)
        if(debug == True):
            print("\n-> feature_reduction\ndata\n")
            pprint(data)
            pprint(vector)
            print("after reduction")
            pprint(reduced)
        return reduced
    def vector(data):
        data = (data_train)
        means, means_feature_wise = (LDA.calc_means(data))
        Sw = (LDA.calc_Sjs(data,means))
        Swi = (LDA.calc_Swinverse(Sw))
        Sb = (LDA.calc_Sb(data,Sw,means_feature_wise))
        A =LDA.A(Swi,Sb)

        point = np.array(data_test)
        reduced_features = 1
        eigenvector ,eigenvalue = (LDA.eigen_vector(A,reduced_features))
        return eigenvector


##### Applying `LDA`, getting eigenvector and Projecting data

In [52]:
eigenvector = LDA.vector(data_train)

reduced_data_train = LDA.feature_reduction(data_train,eigenvector)
reduced_data_train = d3_d2(reduced_data_train)  #2D
target_train = (sum(target_train,[]))           #1D


reduced_data_test = LDA.feature_reduction(data_test,eigenvector)
reduced_data_test = d3_d2(reduced_data_test) 
target_test = (sum(target_test,[]))


`kNN` after `LDA`

In [53]:
data_test = np.array(reduced_data_test) #2D
target_test = np.array(target_test)     #2D
data_test = (data_test.reshape(-1, data_test.shape[-1]))

data_test = data_test.real
kNN = KNeighborsClassifier(n_neighbors=5)
kNN.fit(data_test,target_test)
predicted_label = kNN.predict(np.array(data_test))
acc = (sum([1 for i in range(len(data_test)) if predicted_label[i] == target_test[i]])) / len(data_test)

print("Accuracy with LDA is ",acc)

Accuracy with LDA is  1.0


`kNN` before `LDA`

In [54]:
data_test = (data_test) 
data_test = np.array(data_test) #2D
target_test = np.array(target_test)     #2D
data_test = (data_test.reshape(-1, data_test.shape[-1]))

kNN = KNeighborsClassifier(n_neighbors=5)
kNN.fit(data_test,target_test)
predicted_label = kNN.predict(np.array(data_test))
acc = (sum([1 for i in range(len(data_test)) if predicted_label[i] == target_test[i]]))/len(data_test)

print("Accuracy without LDA is ",acc)

Accuracy without LDA is  1.0


Both are 1.0 , so can't compare here